# Capstone 3 
## Recommendation system
### July 2022

##### project idea

##### https://docs.google.com/document/d/1bybbHktaNtRWpOuZq8H2j6U6X1n7m4VKFeroqADNNYM/edit?usp=sharing
##### project idea
##### https://docs.google.com/document/d/17e3VP4f4s_cnssqMTAmYqmXfuDhb5LBZtTl_VWck-fo/edit?usp=sharing

In [2]:
# import lib

import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns


# Load dataset

In [3]:
# MSDChallengeGettingstarted.pdf
# kaggle_visible_evaluation_triplets.txt
# kaggle_songs.txt
# kaggle_users.txt
# taste_profile_song_to_tracks.txt

In [28]:
basedir="/Users/yuenyeelo/Documents/springboard/projects/Capstone3/msdchallenge/"
songfile=basedir+"kaggle_songs.txt"
usersfile=basedir+"kaggle_users.txt"
profilefile=basedir+"taste_profile_song_to_tracks.txt"
evalfile=basedir+"kaggle_visible_evaluation_triplets.txt"

df_song=pd.read_csv(songfile,names=['song_id', 'id'],sep=' ')
df_users=pd.read_csv(usersfile, names=["user_id"],sep=' ')
df_profile=pd.read_csv(profilefile, names=["song_id", "user_id"],sep=' ')
df_eval=pd.read_csv(evalfile, names=["user_id","song_id","listen_count"],sep='\t')

# EDA 

In [19]:
# songs
print(df_song.isnull().sum())
print(df_song.dtypes)
print(df_song['song_id'].value_counts)
df_song.describe()

song_id    0
id         0
dtype: int64
song_id    object
id          int64
dtype: object
<bound method IndexOpsMixin.value_counts of 0         SOAAADD12AB018A9DD
1         SOAAADE12A6D4F80CC
2         SOAAADF12A8C13DF62
3         SOAAADZ12A8C1334FB
4         SOAAAFI12A6D4F9C66
                 ...        
386208    SOZZZRJ12AB0187A75
386209    SOZZZRV12A8C1361F1
386210    SOZZZSR12AB01854CD
386211    SOZZZWD12A6D4F6624
386212    SOZZZWN12AF72A1E29
Name: song_id, Length: 386213, dtype: object>


,id
count,386213.000000
mean,193107.000000
std,111490.234095
min,1.000000
25%,96554.000000
50%,193107.000000
75%,289660.000000
max,386213.000000


In [15]:
# users
print(df_users.isnull().sum())
print(df_users.dtypes)

df_users.describe()

user_id    0
dtype: int64
user_id    object
dtype: object


,user_id
count,110000
unique,110000
top,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d
freq,1


##  There are total 110,000 users, and 386, 213 songs. There is no nan value.

In [22]:
# df_eval 
print(df_eval.isnull().sum())
print(df_eval.dtypes)

df_eval.describe()

user_id    0
song_id    0
freq       0
dtype: int64
user_id    object
song_id    object
freq        int64
dtype: object


,freq
count,1.450933e+06
mean,3.187149e+00
std,7.051664e+00
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,3.000000e+00
max,9.230000e+02


In [ ]:
#### There is no nan in the eval file.
#### Let see which songs are most popular

In [27]:
df_eval['total_songfreq'] = df_eval.groupby('song_id')['listen_count'].transform('count')
#df_eval.groupby('song_id')
df_eval.head()

,user_id,song_id,freq,total_songfreq
0,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOBONKR12A58A7A7E0,1,4136
1,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOEGIYH12A6D4FC0E3,1,3272
2,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOFLJQZ12A6D4FADA6,1,2668
3,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SOHTKMO12AB01843B0,1,2097
4,fd50c4007b68a3737fe052d5a4f78ce8aa117f3d,SODQZCY12A6D4F9D11,1,177


In [31]:
# cummulative sum of listen count of the songs 
song_grouped = df_eval.groupby(['song_id']).agg({'listen_count':'count'}).reset_index()
song_grouped.head()

,song_id,listen_count
0,SOAAAFI12A6D4F9C66,2
1,SOAAAGK12AB0189572,1
2,SOAAAGQ12A8C1420C8,33
3,SOAAAMT12AB018C9C4,1
4,SOAAAQN12AB01856D3,2


In [34]:
# check popular songs
song_grouped.sort_values(by=['listen_count'],ascending=False)

,song_id,listen_count
38185,SOFRQTD12A81C233C0,5043
5478,SOAUWYT12A81C206F1,4483
10546,SOBONKR12A58A7A7E0,4136
6096,SOAXGDH12A8C13F8A1,3780
121599,SOSXLTC12AF72A7F54,3672
...,...,...
53566,SOIAHHF12A6701CD2D,1
113449,SOROPMV12AB018AB34,1
113450,SOROPNP12A8C13BE06,1
113453,SOROQGG12AB017F628,1


In [36]:
grouped_sum = song_grouped['listen_count'].sum()
song_grouped['percentage'] = (song_grouped['listen_count'] / grouped_sum ) * 100
song_grouped.sort_values(['listen_count', 'song_id'], ascending=[0,1])

,song_id,listen_count,percentage
38185,SOFRQTD12A81C233C0,5043,0.347569
5478,SOAUWYT12A81C206F1,4483,0.308974
10546,SOBONKR12A58A7A7E0,4136,0.285058
6096,SOAXGDH12A8C13F8A1,3780,0.260522
121599,SOSXLTC12AF72A7F54,3672,0.253079
...,...,...,...
163191,SOZZYCA12AB0182579,1,0.000069
163199,SOZZZFM12AC4687A9E,1,0.000069
163200,SOZZZFW12AB0187AD1,1,0.000069
163202,SOZZZKJ12A6D4FBF66,1,0.000069


# Methods

# Prediction